In [ ]:
!apt-get install -y tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,811 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123588 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
!pip install textract
!pip install pandas
!pip install doc2txt
!pip install docx2txt
!pip install PyPDF2
!pip install PyMuPDF
!pip install python-docx
!pip install python-pptx
!pip install pytesseract
!pip install Pillow
!pip install streamlit
!pip install langchain
!pip install huggingface-hub
#!pip install paddlepaddle-gpu==2.5.2.post102 -f https://www.paddlepaddle.org.cn/whl/windows/mkl/avx/stable.html
!pip install chromadb
!pip install gdown==4.6.0
!pip install requests
!pip install beautifulsoup4
!pip install sentence-transformers
!pip install -U langchain-community
!pip install spacy
!python -m spacy download en_core_web_sm

Requested textract from https://files.pythonhosted.org/packages/6b/3e/ac16b6bf28edf78296aea7d0cb416b49ed30282ac8c711662541015ee6f3/textract-1.6.5-py3-none-any.whl has invalid metadata: .* suffix can only be used with `==` or `!=` operators
    extract-msg (<=0.29.*)
                 ~~~~~~~^
Please use pip<24.1 if you need to use this version.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement doc2txt (from versions: none)
ERROR: No matching distribution found for doc2txt
  Preparing metadata (setu

In [ ]:
import pandas as pd
from PIL import Image
import pytesseract
import PyPDF2
import requests
import docx2txt
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import text_splitter
from langchain.text_splitter import CharacterTextSplitter
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pptx
import csv
import os
import gdown
import chromadb
from chromadb import Client
from chromadb.config import Settings
import spacy
import json

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [ ]:
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="/content/chroma_db")

try:
    collection = chroma_client.create_collection("Company_Documents")
except Exception as e:
    print(f"Error creating collection: {e}")
    collection = chroma_client.get_collection("Company_Documents")


In [ ]:
nlp = spacy.load('en_core_web_sm')

def anonymize_names(text):
    doc = nlp(text)
    anonymized_text = text
    for entity in doc.ents:
        print(f"""
        "text": {entity.text},
        "start": {entity.start_char},
        "end": {entity.end_char},
        "label": {entity.label_}
   """)
        if entity.label_ == 'PERSON':
            anonymized_text = anonymized_text.replace(entity.text, 'ANONYMIZED')
    return anonymized_text

print(anonymize_names(f"""Alice and her best friend Bob decided to explore the mysterious forest near their village. Deeper than they had ever gone, they met Charlie, a wise old man, who warned them of the forest's magic. Despite Bob's caution, Alice's curiosity led them to Daisy, a mischievous fairy. Daisy helped them find their way home, teaching them the value of bravery and caution along the way."""))


        "text": Alice,
        "start": 0,
        "end": 5,
        "label": PERSON
   

        "text": Bob,
        "start": 26,
        "end": 29,
        "label": PERSON
   

        "text": Charlie,
        "start": 132,
        "end": 139,
        "label": PERSON
   

        "text": Bob,
        "start": 204,
        "end": 207,
        "label": PERSON
   

        "text": Alice,
        "start": 219,
        "end": 224,
        "label": PERSON
   

        "text": Daisy,
        "start": 249,
        "end": 254,
        "label": PERSON
   

        "text": Daisy,
        "start": 277,
        "end": 282,
        "label": PERSON
   
ANONYMIZED and her best friend ANONYMIZED decided to explore the mysterious forest near their village. Deeper than they had ever gone, they met ANONYMIZED, a wise old man, who warned them of the forest's magic. Despite ANONYMIZED's caution, ANONYMIZED's curiosity led them to ANONYMIZED, a mischievous fairy. ANONYMIZED helped them find their way hom

In [ ]:
# Define the embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to download files from Google Drive using gdown
def download_files_from_drive(folder_id, local_folder):
    os.makedirs(local_folder, exist_ok=True)
    gdown.download_folder(id=folder_id, output=local_folder, quiet=False, use_cookies=False)
    upload_files_in_directory(local_folder)

def extract_content_from_file(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()

    if file_extension == '.pdf':
        return extract_content_from_pdf(file_path)
    elif file_extension == '.xlsx':
        return extract_content_from_xlsx(file_path)
    elif file_extension == '.csv':
        return extract_content_from_csv(file_path)
    elif file_extension == '.docx':
        return extract_content_from_docx(file_path)
    elif file_extension == '.txt':
        return extract_content_from_txt(file_path)
    elif file_extension in ['.jpg', '.jpeg', '.png']:
        return extract_content_from_image(file_path)
    elif file_extension == '.pptx':
        return extract_content_from_pptx(file_path)
    else:
        raise ValueError(f"Unsupported file type: {file_extension}")

def extract_content_from_pdf(file_path):
    content = ""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        for page in reader.pages:
            content += page.extract_text()
    return content

def extract_content_from_xlsx(file_path):
    df = pd.read_excel(file_path)
    return df.to_csv(index=False)

def extract_content_from_csv(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def extract_content_from_docx(file_path):
    return docx2txt.process(file_path)

def extract_content_from_txt(file_path):
    with open(file_path, 'r') as file:
        return file.read()

def extract_content_from_image(file_path):
    image = Image.open(file_path)
    return pytesseract.image_to_string(image)

def extract_content_from_pptx(file_path):
    prs = pptx.Presentation(file_path)
    content = ""
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                content += shape.text + "\n"
    return content

def extract_links_from_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    links = []
    for a_tag in soup.find_all('a', href=True):
        links.append(a_tag['href'])
    return links

def extract_content_from_url(url):
    try:
        response = requests.get(url)
        content_type = response.headers['Content-Type'].lower()
        if 'pdf' in content_type:
            return extract_content_from_pdf(url)
        elif 'image' in content_type:
            return extract_content_from_image(url)
        else:
            return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

def process_links(text):
    links = extract_links_from_text(text)
    linked_content = ""
    for link in links:
        linked_content += extract_content_from_url(link)
    return linked_content

def chunk_and_embed_text(text):
    # Initialize the text splitter
    text_splitter = CharacterTextSplitter(chunk_size=128, chunk_overlap=64)

    # Split text into chunks
    chunks = text_splitter.split_text(text)

    # Embed each chunk
    embeddings = embedding_model.encode(chunks)

    # Convert embeddings from ndarray to list
    embeddings = [embedding.tolist() for embedding in embeddings]

    return chunks, embeddings

def upload_file_to_db(file_path, folder_name, access_role):
    content = extract_content_from_file(file_path)

    #Testing name anonymization
    if 'Meeting Transcripts' in file_path:
      content = anonymize_names(content)

    linked_content = process_links(content)
    full_content = content + linked_content
    chunks, embeddings = chunk_and_embed_text(full_content)
    document_id = os.path.basename(file_path)

    # Tag the document with metadata
    metadata = {
        'folder_name': folder_name,
        'file_type': os.path.splitext(file_path)[1].lower(),
        'access_role': access_role
    }

    # Add document and its embeddings to the collection if not already present
    for i, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
        unique_id = f"{document_id}_{i}"
        existing_ids = collection.get(ids=[unique_id])
        if not existing_ids['ids']:
            collection.add(ids=[unique_id], documents=[chunk], embeddings=[embedding], metadatas=[metadata])
        else:
            print(f"Skipping existing ID: {unique_id}")

def upload_files_in_directory(directory_path):
    for root, _, files in os.walk(directory_path):
        folder_name = os.path.basename(root)
        for file in files:
            file_path = os.path.join(root, file)
            access_role = 'Executive Access' if 'Contratcs' in file_path or 'Business Data' in file_path else 'General Access'
            try:
                upload_file_to_db(file_path, folder_name, access_role)
                print(f"Uploaded: {file_path} with role: {access_role}")
            except ValueError as e:
                print(e)








# Replace with the ID of your Google Drive folder
drive_folder_id = '1bFgwH3rvmZ6BD_Rl9sfMjB2MDtkZbjIy'
local_download_folder = 'downloaded_files'

# Download and process files
download_files_from_drive(drive_folder_id, local_download_folder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Uploaded: downloaded_files/Marketing Posters/Automation Platform.png with role: General Access


Uploaded: downloaded_files/Marketing Posters/AiDIN.jpg with role: General Access
Uploaded: downloaded_files/Contratcs/Attach_rates.csv with role: Executive Access


Streaming output truncated to the last 5000 lines.
,
        "start": 13330,
        "end": 13338,
        "label": PERSON
   

        "text": Stephen
,
        "start": 13578,
        "end": 13586,
        "label": PERSON
   

        "text": Jerry Norman,
        "start": 13594,
        "end": 13606,
        "label": PERSON
   

        "text": Daryl,
        "start": 13660,
        "end": 13665,
        "label": PERSON
   

        "text": Stephen
We,
        "start": 13725,
        "end": 13735,
        "label": PERSON
   

        "text": 0:12:14.100,
        "start": 13829,
        "end": 13840,
        "label": CARDINAL
   

        "text": Stephen
We,
        "start": 13864,
        "end": 13874,
        "label": PERSON
   

        "text": today,
        "start": 13907,
        "end": 13912,
        "label": DATE
   

        "text": 0:12:24.520,
        "start": 14012,
        "end": 14023,
        "label": CARDINAL
   

        "text": Ford,
        "start": 14024,
        



        "text": Ford,
        "start": 55630,
        "end": 55634,
        "label": ORG
   

        "text": Daryl,
        "start": 55636,
        "end": 55641,
        "label": PERSON
   

        "text": Stephen
Actually,
        "start": 55683,
        "end": 55699,
        "label": PERSON
   

        "text": 0:51:41.610,
        "start": 55763,
        "end": 55774,
        "label": CARDINAL
   

        "text": Stephen
I,
        "start": 55782,
        "end": 55791,
        "label": PERSON
   

        "text": Stephen
It's,
        "start": 56014,
        "end": 56026,
        "label": PERSON
   

        "text": 0:51:53.920,
        "start": 56194,
        "end": 56205,
        "label": CARDINAL
   

        "text": Stephen,
        "start": 56228,
        "end": 56235,
        "label": GPE
   

        "text": Ford,
        "start": 56404,
        "end": 56408,
        "label": ORG
   

        "text": Daryl,
        "start": 56410,
        "end": 56415,
        "label": P

Uploaded: downloaded_files/Meeting Transcripts/Technology Council Meeting_2023-11-09 Transcript.docx with role: General Access
Uploaded: downloaded_files/Business Data/Alteryx US Business Insights Q2 2022 Release Docs.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/rsm-alteryx-services.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/Regional KPI Report - North America.pptx with role: Executive Access
Uploaded: downloaded_files/Business Data/alteryxuipath-solution-briefv2-final.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/alteryx-promote-datasheet.pdf with role: Executive Access
Uploaded: downloaded_files/Business Data/Alteryx - 24 Best Practices.pdf with role: Executive Access
Uploaded: downloaded_files/Customer Report/Alteryx_Smart_Foods.pdf with role: General Access
Uploaded: downloaded_files/Customer Report/Healthy_Foods_Inc..pdf with role: General Access
Uploaded: downloaded_files/Customer Report/Tech_Gad